In [53]:
from pandas_datareader import data as wb

SPY = wb.DataReader('SPY', data_source='yahoo', start='1999-1-1')
SPY.to_csv('SPY'+'.csv')

In [54]:
SPY

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1998-12-31,123.937500,122.468750,123.312500,123.312500,6790500.0,82.680634
1999-01-04,125.218750,121.718750,123.375000,123.031250,9450400.0,82.492058
1999-01-05,124.875000,122.937500,122.937500,124.437500,8031000.0,83.434944
1999-01-06,127.750000,125.750000,125.812500,127.437500,7737700.0,85.446434
1999-01-07,127.218750,125.781250,126.375000,126.812500,5504900.0,85.027359
...,...,...,...,...,...,...
2020-10-26,342.980011,335.619995,342.130005,339.390015,91473000.0,339.390015
2020-10-27,340.119995,337.989990,339.760010,338.220001,65994100.0,338.220001
2020-10-28,338.250000,326.130005,332.100006,326.660004,127094300.0,326.660004


In [50]:
from datetime import datetime, timedelta
import math

def strategy_monthly(volum, SPY_sum, months_to_breakeven, high_dt, high):
    for i in range(months_to_breakeven):
        dt = high_dt + timedelta(days=(30.4*(i+1)))
        dt_str = dt.strftime('%Y-%m-%d')
        
        while not dt_str in SPY.index:
            dt = dt + timedelta(days=1)
            dt_str = dt.strftime('%Y-%m-%d')
        SPY_sum += SPY.loc[dt_str]['Close']
        volum += 1
    
    return (SPY_sum, volum)

def strategy_percentage(volum, SPY_sum, months_to_breakeven, high_dt, high):
    tmp = high
    weight = 2.5
    for index, row in SPY.iterrows():
        if row['Close'] < tmp * 0.9:
            tmp = tmp * 0.9
            SPY_sum += row['Close'] * weight
            volum += weight
            weight += 0.5
        if row['Close'] > tmp * 1.1:
            tmp = tmp * 1.1
            SPY_sum += row['Close'] * weight
            volum += weight
            weight -= 0.5
        if tmp > high:
            break
        
    return (SPY_sum, volum)

def strategy_monthly_dynamic(volum, SPY_sum, months_to_breakeven, high_dt, high):
    for i in range(months_to_breakeven):
        dt = high_dt + timedelta(days=(30.4*(i+1)))
        dt_str = dt.strftime('%Y-%m-%d')

        while not dt_str in SPY.index:
            dt = dt + timedelta(days=1)
            dt_str = dt.strftime('%Y-%m-%d')

        val = SPY.loc[dt_str]['Close']
        diff_percent = (high - val)/high
        weight = 0.5
        weight = weight*(1+diff_percent*4)

        SPY_sum += val*weight
        volum += weight
    
    return (SPY_sum, volum)

In [51]:
def find_adjust_breakeven(start_dt, low_dt, s, v):
    avg_price = s/v
    print('sum:' + str(s) + ', volum:' + str(v) + ', avg price:' + str(avg_price))
    for index, row in SPY.iterrows():
        if index <= low_dt:
            continue
        if row['Close'] > avg_price:
            months_to_adj_breakeven = index - start_dt
            months_to_adj_breakeven = math.ceil(months_to_adj_breakeven.days/30.4)
            return months_to_adj_breakeven

def analyze_crisis(msg, start_time):
    print('========== analyzing ' + msg + ' ' + start_time + ' ========== ')
    
    high = SPY.loc[start_time]['Close'] #2008 最高点开始套牢
    print('start: ' + str(high))
    start_dt = datetime.strptime(start_time, '%Y-%m-%d')
    
    low_dt = start_dt
    low = high
    breakeven_dt = start_dt
    flag_crisis_start = False
    for index, row in SPY.iterrows():
        if index <= start_dt:
            continue
        flag_crisis_start = True
        val = row['Close']
        if val < low:
            low = val
            low_dt = index
        if val > high and flag_crisis_start:
            print(val)
            breakeven_dt = index
            break
    
    print('low: ' + str(low))
    loss = (low - high)/high
    print('loss: ' + str(loss))
    
    months_to_low = low_dt - start_dt
    months_to_low = math.ceil(months_to_low.days/30.4)
    print("months to low: " + str(months_to_low))

    months_to_breakeven = breakeven_dt - start_dt
    months_to_breakeven = math.ceil(months_to_breakeven.days/30.4)
    print("months to breakeven: " + str(months_to_breakeven)+'\r\n')
    
    # assuming initial invest is equal to 5 months
    volum = 5
    SPY_sum = volum * high
    
    print('strategy monthly => ')
    s, v = strategy_monthly(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven = find_adjust_breakeven(start_dt, low_dt, s, v)
    print("months to adj breakeven: " + str(months_to_adj_breakeven)+'\r\n')
    
    print('strategy percentage => ')
    s2, v2 = strategy_percentage(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven2 = find_adjust_breakeven(start_dt, low_dt, s2, v2)
    print("months to adj breakeven: " + str(months_to_adj_breakeven2)+'\r\n')
    
    print('strategy percentage dynamic => ')
    s3, v3 = strategy_monthly_dynamic(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven3 = find_adjust_breakeven(start_dt, low_dt, s3, v3)
    print("months to adj breakeven: " + str(months_to_adj_breakeven3)+'\r\n')
    
analyze_crisis('2008 crisis', start_time='2007-10-12')

========== analyzing 2008 crisis 2007-10-12 ========== 
start: 156.3300018310547
156.72999572753906
low: 68.11000061035156
loss: -0.5643190698356301
months to low: 17
months to breakeven: 66

strategy monthly => 
sum:8826.530052185059, volum:71, avg price:124.3173246786628
months to adj breakeven: 38

strategy percentage => 
sum:20565.053062438965, volum:192.0, avg price:107.10965136686961
months to adj breakeven: 24

strategy percentage dynamic => 
sum:8039.779179751698, volum:67.07823260024186, avg price:119.8567533474743
months to adj breakeven: 31



In [52]:
analyze_crisis('2000 crisis', start_time='2000-09-1')

========== analyzing 2000 crisis 2000-09-1 ========== 
start: 152.5
152.6199951171875
low: 78.0999984741211
loss: -0.48786886246477973
months to low: 26
months to breakeven: 81

strategy monthly => 
sum:10216.948112487793, volum:86, avg price:118.80172223823016
months to adj breakeven: 52

strategy percentage => 
sum:8604.942478179932, volum:75.0, avg price:114.73256637573242
months to adj breakeven: 41

strategy percentage dynamic => 
sum:9645.775947254871, volum:83.50723786901253, avg price:115.50826243810155
months to adj breakeven: 41

